In [1]:
import os
import h5py
import time
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import glob
import torchmetrics
from torch.fft import fft, rfft, ifft
# import librosa
import itertools
import numpy as np
import pandas as pd
import multiprocessing
from multiprocessing import Pool
from scipy import signal
from numba import jit, prange
import matplotlib.pyplot as plt
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader
# from nnAudio.Spectrogram import CQT1992v2, STFT
from numba.experimental import jitclass
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from pytorch_lightning.loggers import WandbLogger
from sklearn.metrics import classification_report
from pytorch_lightning.callbacks import ModelCheckpoint
#torch.multiprocessing.set_start_method('spawn')# good solution !!!!

import wandb
# wandb.login()
os.environ["WANDB_MODE"] = 'disabled'

import os
os.environ["MKL_THREADING_LAYER"] = "GNU"

In [2]:
def normalize(time_series):
        
    # Scaling the data in the [-1,1] range so as to transform to polar co-ordinates
    # for Gramian angular fields.

    min_stamp = np.amin(time_series)
    max_stamp = np.amax(time_series)
    time_series = (2*time_series - max_stamp - min_stamp)/(max_stamp - min_stamp)

    # Checking for any floating point accuracy.
    time_series = np.where(time_series >= 1., 1., time_series)
    time_series = np.where(time_series <= -1., -1., time_series)

    return time_series

def whiten(x):
    # Assuming x has shape (length, channels)
    hann = torch.hann_window(x.shape[0], periodic=True, dtype=torch.float32)
    whitened_channels = []
    
    for ch in range(x.shape[1]):
        single_channel_data = torch.tensor(x[:, ch], dtype=torch.float32)
        spec = fft(single_channel_data * hann)
        mag = torch.sqrt(torch.real(spec * torch.conj(spec)))
        whitened_channel = ifft(spec / mag) * torch.sqrt(torch.tensor(len(single_channel_data) / 2, dtype=torch.float32))
        whitened_channels.append(whitened_channel.real)
        
    return torch.stack(whitened_channels, dim=1)


def apply_bandpass(x, lf=20, hf=500, order=8, sr=2048):
    # Generate the filter coefficients
    sos = signal.butter(order, [lf, hf], btype='bandpass', output='sos', fs=sr)
    normalization = np.sqrt((hf - lf) / (sr / 2))

    # Apply the filter
    if len(x.shape) == 1:
        # Single channel
        x *= signal.tukey(x.shape[0], 0.2)
        x = signal.sosfiltfilt(sos, x) / normalization
    else:
        # Multi-channel
        num_channels = x.shape[0]
        for ch in range(num_channels):
            x[ch, :] *= signal.tukey(x[ch, :].shape[0], 0.2)
            x[ch, :] = signal.sosfiltfilt(sos, x[ch, :]) / normalization

    return x

In [3]:
# Load CSV file into a DataFrame
DATA_DIR = "/data/rgura001/ML4GWsearch/g2net-gravitational-wave-detection"
labels_df = pd.read_csv(DATA_DIR+'/training_labels.csv')

In [4]:
# Use glob to find all the .npy files in all subfolders
file_paths = glob.glob(DATA_DIR+'/train/*/*/*/*.npy')

# Sort the files
file_paths = sorted(file_paths)
# Extract filenames from paths and remove '.npy'
file_names = [fp.split('/')[-1].replace('.npy', '') for fp in file_paths]


In [5]:
files_df = pd.DataFrame({'id': file_names, 'filepath': file_paths})
merged_df = pd.merge(files_df, labels_df, on='id')
merged_df.head()

,id,filepath,target
0,00000e74ad,/data/rgura001/ML4GWsearch/g2net-gravitational...,1
1,00001f4945,/data/rgura001/ML4GWsearch/g2net-gravitational...,0
2,0000661522,/data/rgura001/ML4GWsearch/g2net-gravitational...,0
3,00007a006a,/data/rgura001/ML4GWsearch/g2net-gravitational...,0
4,0000a38978,/data/rgura001/ML4GWsearch/g2net-gravitational...,1


In [6]:
test_file_paths = glob.glob(DATA_DIR+'/test/*/*/*/*.npy')

# Sort the files
test_file_paths = sorted(test_file_paths)
# Extract filenames from paths and remove '.npy'
test_file_names = [fp.split('/')[-1].replace('.npy', '') for fp in test_file_paths]
test_files_df = pd.DataFrame({'id': test_file_names, 'filepath': test_file_paths})

In [7]:
class G2NetDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        filepath = self.dataframe.iloc[idx]['filepath']
        label = self.dataframe.iloc[idx]['target']
        
        # Load .npy file
        data = np.load(filepath)
        
        data = normalize(data)
        #data = whiten(data)
        data = apply_bandpass(data)
        # Convert to PyTorch tensor
        data = torch.tensor(data, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)

        # print(data.shape, label.shape)
        
        return data, label

In [8]:
class G2NetTestDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        filepath = self.dataframe.iloc[idx]['filepath']
        
        # Load .npy file
        data = np.load(filepath)
        
        data = normalize(data)
        #data = whiten(data)
        data = apply_bandpass(data)
        # Convert to PyTorch tensor
        data = torch.tensor(data, dtype=torch.float32)
        
        return data

In [9]:
dataset = G2NetDataset(merged_df)
test_dataset = G2NetTestDataset(test_files_df)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size


# Randomly split dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for train and validation datasets
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=16)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=16)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=16)

In [10]:


class Encoder(pl.LightningModule):

    def __init__(self, input_shape=(4096, 3)):
        super(Encoder, self).__init__()
        
        # Define your architecture here
        self.conv1 = nn.Conv1d(input_shape[1], 128, kernel_size=5, padding=2)
        self.instance_norm1 = nn.InstanceNorm1d(128)
        self.prelu1 = nn.PReLU()
        self.dropout1 = nn.Dropout(0.2)
        
        self.conv2 = nn.Conv1d(128, 256, kernel_size=11, padding=5)
        self.instance_norm2 = nn.InstanceNorm1d(256)
        self.prelu2 = nn.PReLU()
        self.dropout2 = nn.Dropout(0.2)
        
        self.conv3 = nn.Conv1d(256, 512, kernel_size=21, padding=10)
        self.instance_norm3 = nn.InstanceNorm1d(512)
        self.prelu3 = nn.PReLU()
        self.dropout3 = nn.Dropout(0.2)
        
        self.attention_data = nn.Conv1d(512, 256, kernel_size=1)
        self.attention_softmax = nn.Conv1d(512, 256, kernel_size=1)
        
        self.fc1 = nn.Linear(256 * input_shape[0] // 4, 512)
        self.instance_norm4 = nn.InstanceNorm1d(512)
        self.fc2 = nn.Linear(512, 1)
        
        self.train_auc = torchmetrics.AUROC(task="binary")
        self.val_auc = torchmetrics.AUROC(task="binary")

        
    def forward(self, x):
        x1 = self.prelu1(self.instance_norm1(self.conv1(x)))
        x1 = self.dropout1(x1)
        x1 = F.max_pool1d(x1, kernel_size=2)
        
        x2 = self.prelu2(self.instance_norm2(self.conv2(x1)))
        x2 = self.dropout2(x2)
        x2 = F.max_pool1d(x2, kernel_size=2)
        
        conv3 = self.prelu3(self.instance_norm3(self.conv3(x2)))
        conv3 = self.dropout3(conv3)
        
        attention_data = self.attention_data(conv3)
        attention_softmax = F.softmax(self.attention_softmax(conv3), dim=2)
        
        multiply_layer = attention_data * attention_softmax
        dense_layer = F.sigmoid(self.instance_norm4(self.fc1(multiply_layer.view(multiply_layer.size(0), -1))))
        
        output_layer = self.fc2(dense_layer)
        return output_layer


    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.float()  # Ensure y is a float
        y_hat = self(x)
        y_hat = y_hat.view(-1)  # Change shape from [batch_size, 1] to [batch_size]
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        self.log('train_loss', loss)
        self.train_auc(y_hat.sigmoid(), y.int())
        self.log('train_auc', self.train_auc, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.float()  # Ensure y is a float
        y_hat = self(x)
        y_hat = y_hat.view(-1)  # Change shape from [batch_size, 1] to [batch_size]
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        self.log('val_loss', loss)
        self.val_auc(y_hat.sigmoid(), y.int())
        self.log('val_auc', self.val_auc, on_step=True, on_epoch=True)
        return loss
    
    def predict_step(self, batch, batch_idx):
        x = batch
        y_hat = self(x)
        y_hat = y_hat.view(-1) 
        y_pred_prob = y_hat.sigmoid() 
        return y_pred_prob


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-5)

In [ ]:
if __name__ == '__main__':
    
    wandb_logger = WandbLogger(project='G2Net', log_model='all')
    
    # Define ModelCheckpoint callback
    checkpoint_callback = ModelCheckpoint(
        monitor='val_auc',   # Replace 'val_auc' with the actual name of the metric you want to monitor
        dirpath='checkpoints/v2/',
        filename='best-checkpoint',
        save_top_k=1,
        mode='max'
    )
    model = Encoder(input_shape=(4096, 3))
    trainer = pl.Trainer(
        logger=wandb_logger,
        callbacks=[checkpoint_callback],
        max_epochs=10,
        accelerator="gpu", 
        devices=1,
        accumulate_grad_batches=2,
        log_every_n_steps=1,
        enable_progress_bar = True,
    )

    # Train the model
    trainer.fit(model, train_loader, val_loader)

In [16]:
model = Encoder(input_shape=(4096, 3))
trained_model = model.load_from_checkpoint(checkpoint_path='checkpoints/v2/best-checkpoint.ckpt')
trained_model.eval()

Encoder(
  (conv1): Conv1d(3, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (instance_norm1): InstanceNorm1d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (prelu1): PReLU(num_parameters=1)
  (dropout1): Dropout(p=0.2, inplace=False)
  (conv2): Conv1d(128, 256, kernel_size=(11,), stride=(1,), padding=(5,))
  (instance_norm2): InstanceNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (prelu2): PReLU(num_parameters=1)
  (dropout2): Dropout(p=0.2, inplace=False)
  (conv3): Conv1d(256, 512, kernel_size=(21,), stride=(1,), padding=(10,))
  (instance_norm3): InstanceNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (prelu3): PReLU(num_parameters=1)
  (dropout3): Dropout(p=0.2, inplace=False)
  (attention_data): Conv1d(512, 256, kernel_size=(1,), stride=(1,))
  (attention_softmax): Conv1d(512, 256, kernel_size=(1,), stride=(1,))
  (fc1): Linear(in_features=262144, out_features=512, bias=True)
  (instan

In [17]:
trainer = pl.Trainer(devices=1)
predictions= trainer.predict(trained_model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/kpal002/miniconda/envs/gwsearchenv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting: 0it [00:00, ?it/s]

In [18]:
flat_predictions = [item for sublist in predictions for item in sublist.cpu().numpy()]

In [19]:
submission_df = pd.read_csv(DATA_DIR+'/sample_submission.csv')
submission_df['target'] = flat_predictions
submission_df.to_csv('submission.csv', index=False)

In [ ]:
true_labels, val_predictions=trainer.predict(trained_model, val_loader)

In [ ]:
val_flat_predictions = [item for sublist in val_predictions for item in sublist.cpu().numpy()]

In [ ]:
true_labels_list = []
for data, labels in val_dataset:
    true_labels_list.append(labels)


In [ ]:
probs_tensor = torch.tensor(val_flat_predictions)
# Convert probabilities to predicted labels
predicted_labels = (probs_tensor > 0.5).float()

# Compute accuracy
true_labels = torch.tensor(true_labels_list)  # Assuming true_labels_list is your list of true labels
correct = (predicted_labels == true_labels).float().sum()
accuracy = correct / len(true_labels)
print(f"Accuracy: {accuracy.item():.4f}")

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(true_labels_list, predicted_labels)

# Normalize by row (i.e., by the number of samples in each true class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm_percentage = cm_normalized * 100

plt.figure(figsize=(8, 8))
plt.imshow(cm_percentage, interpolation='nearest', cmap=plt.cm.viridis)  # Using viridis colormap
plt.title('Normalized Confusion Matrix', fontsize=16)
plt.colorbar(label='Percentage %')
tick_marks = np.arange(2)  # Assuming binary classification
plt.xticks(tick_marks, ['Class 0', 'Class 1'], rotation=45, fontsize=12)
plt.yticks(tick_marks, ['Class 0', 'Class 1'], fontsize=12)

thresh = cm_percentage.max() / 2.
for i, j in itertools.product(range(cm_percentage.shape[0]), range(cm_percentage.shape[1])):
    plt.text(j, i, f"{cm_percentage[i, j]:.2f}%",
             horizontalalignment="center",
             fontsize=14,
             color="white" if cm_percentage[i, j] > thresh else "black")

plt.ylabel('True label', fontsize=14)
plt.xlabel('Predicted label', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Compute the ROC curve
fpr, tpr, thresholds = roc_curve(true_labels_list, val_flat_predictions)

# Compute AUC
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
report = classification_report(true_labels_list, predicted_labels)
print(report)